In [2]:
# 資料中提供詳細的全台藥局資訊，包含藥局名稱、地址、電話、營業時間等相關基本訊息，
# 以及各藥局的note與成人和孩童的口罩數量，還有藥局所在的經緯度資訊。
# 延伸應用：藥局口罩庫存查詢、補充庫存提醒、庫存地圖視覺化、預測口罩需求、預約系統


import requests
from requests import get
import json

# json.dumps()
# 將python物件轉成json字串，返回type為str
# 從python物件轉換為json字串

# json.loads()
# 將json字串，返回python物件，返回type為dict
# 從json字串轉換為python物件



# 利用 requests 對 API 來源發送一個請求
url = 'https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json'
response = requests.get(url)

# 將請求回應的內容存成一個字串格式
dic = response.text


# 將長得像 json 格式的字串解析成字典或列表
#python json字串轉成python物件需要使用json模塊的loads()
data = json.loads(dic)
#print(data)


med_count = {} #建立一個空字典med_count
print(med_count) #印出空字典

# 填入欄位名稱
for dic in data['features']:# 從 data['features'] 取出一個 dic 資料
    conunty = dic['properties']['county'] #取出字典的properties與county欄位
#    print(conunty)
    if conunty not in med_count: #如果county不在med_count中
         med_count[conunty] = 0 #則將county加入med_count,設為0
    else:
        med_count[conunty] += 1 #如果county在med_count中，則med_count[county] + 1
print(med_count)

# {'臺北市': 338, '': 48, '高雄市': 421, '臺中市': 419, '臺南市': 270, '基隆市': 56, '新竹市': 38, '嘉義市': 66, '新北市': 500, '桃園市': 263, '新竹縣': 45, '宜蘭縣': 75, '苗栗縣': 55, '彰化縣': 178, '南投縣': 66, '雲林縣': 128, '嘉義縣': 83, '屏東縣': 139, '澎湖縣': 10, '花蓮縣': 45, '臺東縣': 22, '金門縣': 5, '連江縣': 0}


#---4. 接下來請你計算出每個地區的成人剩餘口罩數量，並且將結果從大到小排列，完成後 Commit 第三個版本：
mask_count = {}

# 填入欄位名稱
for d in data['features']:
    conunty = d['properties']['county'] #取出字典的properties與county欄位
    adult_mask_count = d['properties']['mask_adult'] #取出字典的properties與mask_adult（成人口罩) 欄位

    if conunty not in mask_count: #如果county不在mask_count中
      mask_count[conunty] = adult_mask_count #則將county加入mask_count,設為adult_mask_count, 不會變了
    else:
      mask_count[conunty] += adult_mask_count #如果county在mask_count中，則mask_count[county] + adult_mask_count,持續累加總數

# 將結果從大到小排列
med_count = dict(sorted(mask_count.items(), key=lambda item: item[1]))

print(mask_count)
# {'台北市': 12345, '新北市': 45678 ...}


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [3]:
import sqlite3
import datetime
import requests
import json

#計算的「各地區的藥局數量」存入資料庫的 `pharmacies` 表格，欄位包含「地區」、「數量」、「新增時間」
conn = sqlite3.connect('example.db')
#這行創建了一個 SQLite 資料庫連接物件 (conn)，並連接到名為 example.db 的 SQLite 資料庫。
#如果 example.db 不存在，則會在同一個目錄下創建一個新的資料庫。
c = conn.cursor()
#創建了一個 cursor 物件 c，它可以用來執行 SQL 查詢和操作資料庫。conn.cursor() 是一個方法
#，用於建立一個 cursor 物件，這個 cursor 物件可以用來執行 SQL 查詢並處理查詢結果。

# 新增pharmacies表格且清空資料表
c.execute('''CREATE TABLE IF NOT EXISTS pharmacies
             (city text, counts text, createdAt datetime)''')
c.execute('''DELETE FROM pharmacies''')
conn.commit()

# 新增資料

# 利用 requests 對 API 來源發送一個請求
url = 'https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json'
response = requests.get(url)

# 將請求回應的內容存成一個字串格式
dic = response.text


# 將長得像 json 格式的字串解析成字典或列表
#python json字串轉成python物件需要使用json模塊的loads()
data = json.loads(dic)
#print(data)

#3.計算各地區的藥局數量-------------------------
med_count = {} #建立一個空字典med_count
print(med_count) #印出空字典

# 填入欄位名稱
for dic in data['features']:# 從 data['features'] 取出一個 dic 資料
    conunty = dic['properties']['county'] #取出字典的properties的county欄位
#    print(conunty)
    if conunty not in med_count: #如果county不在med_count中
         med_count[conunty] = 0 #則將county加入med_count,設為0,ex:med_count = {"台北市": 0, "新北市": 1}
         #dictionary[key] = value
    else:
        med_count[conunty] += 1 #如果county在med_count中，則med_count[county] + 1
print
(med_count)


for city, counts in med_count.items():
  #Your Code # item()把將每一筆Med_count資料的Key, value分別取出來新增到資料庫
  t = datetime.datetime.now()
  print(f"INSERT INTO stocks VALUES ('{city}', {counts}, '{t}')")
  c.execute(f"INSERT INTO pharmacies VALUES ('{city}', {counts}, '{t}')")
  conn.commit()

# 查詢資料
c.execute("SELECT * FROM pharmacies")
print(c.fetchall())

conn.commit()
conn.close()

{}
INSERT INTO stocks VALUES ('臺北市', 338, '2024-04-08 06:39:01.892124')
INSERT INTO stocks VALUES ('', 48, '2024-04-08 06:39:01.901375')
INSERT INTO stocks VALUES ('高雄市', 421, '2024-04-08 06:39:01.912796')
INSERT INTO stocks VALUES ('臺中市', 419, '2024-04-08 06:39:01.922101')
INSERT INTO stocks VALUES ('臺南市', 270, '2024-04-08 06:39:01.931375')
INSERT INTO stocks VALUES ('基隆市', 56, '2024-04-08 06:39:01.941212')
INSERT INTO stocks VALUES ('新竹市', 38, '2024-04-08 06:39:01.949218')
INSERT INTO stocks VALUES ('嘉義市', 66, '2024-04-08 06:39:01.956403')
INSERT INTO stocks VALUES ('新北市', 500, '2024-04-08 06:39:01.964391')
INSERT INTO stocks VALUES ('桃園市', 263, '2024-04-08 06:39:01.972534')
INSERT INTO stocks VALUES ('新竹縣', 45, '2024-04-08 06:39:01.981473')
INSERT INTO stocks VALUES ('宜蘭縣', 75, '2024-04-08 06:39:01.989332')
INSERT INTO stocks VALUES ('苗栗縣', 55, '2024-04-08 06:39:01.999264')
INSERT INTO stocks VALUES ('彰化縣', 178, '2024-04-08 06:39:02.008184')
INSERT INTO stocks VALUES ('南投縣', 66, '20